In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import re
import os
import io
import time
import random

from sklearn.model_selection import train_test_split

print(tf.__version__)

2.3.0


In [2]:
path_to_file = os.getenv('HOME')+"/aiffel/scoring_chatbot/ChatbotData.csv"

corpus = pd.read_csv(path_to_file, encoding = 'utf-8')

print("Data Size:", len(corpus))
print("Example:")

for sen in corpus[0:100][::20]: print(">>", sen)

Data Size: 11823
Example:
>> Q
>> A
>> label


In [3]:
corpus.iloc[:5, :]

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
questions = corpus['Q']
questions

0                         12시 땡!
1                    1지망 학교 떨어졌어
2                   3박4일 놀러가고 싶다
3                3박4일 정도 놀러가고 싶다
4                        PPL 심하네
                  ...           
11818             훔쳐보는 것도 눈치 보임.
11819             훔쳐보는 것도 눈치 보임.
11820                흑기사 해주는 짝남.
11821    힘든 연애 좋은 연애라는게 무슨 차이일까?
11822                 힘들어서 결혼할까봐
Name: Q, Length: 11823, dtype: object

In [5]:
answers = corpus['A']
answers

0                      하루가 또 가네요.
1                       위로해 드립니다.
2                     여행은 언제나 좋죠.
3                     여행은 언제나 좋죠.
4                      눈살이 찌푸려지죠.
                   ...           
11818          티가 나니까 눈치가 보이는 거죠!
11819               훔쳐보는 거 티나나봐요.
11820                      설렜겠어요.
11821    잘 헤어질 수 있는 사이 여부인 거 같아요.
11822          도피성 결혼은 하지 않길 바라요.
Name: A, Length: 11823, dtype: object

In [6]:
print(questions[:5], answers[:5])

0             12시 땡!
1        1지망 학교 떨어졌어
2       3박4일 놀러가고 싶다
3    3박4일 정도 놀러가고 싶다
4            PPL 심하네
Name: Q, dtype: object 0     하루가 또 가네요.
1      위로해 드립니다.
2    여행은 언제나 좋죠.
3    여행은 언제나 좋죠.
4     눈살이 찌푸려지죠.
Name: A, dtype: object


In [7]:
def preprocess_sentence(sentence):
    sentence = sentence.lower()

#     sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z가-힣0-9?.!,]+", " ", sentence)

    sentence = sentence.strip()
    
    return sentence
print("슝=3")

슝=3


In [8]:
preprocess_sentence("하루가 또 가네요. #$%*^( 하하ㅏㅁAFDH")

'하루가 또 가네요. 하하 afdh'

In [9]:
# src_corpus = []
# tgt_corpus = []
# for s, t in zip(questions[:5], answers[:5]):
#     print(len(s), s)
#     if len(s) < 40 and len(t) < 40:
#         cleaned_src = preprocess_sentence(s)
# #         print(cleaned_src)
#         cleaned_tgt = preprocess_sentence(t)
# #         print(cleaned_tgt)
#         src_corpus.append(mecab.morphs(cleaned_src))
# #         print(src_corpus)
#         tgt_corpus.append(mecab.morphs(cleaned_tgt))
# #         print(tgt_corpus)
# print(src_corpus[0])

In [10]:
# from konlpy.tag import Mecab
# mecab = Mecab()

# def build_corpus(src, tgt):
    
#     # 전처리, 토큰화, 길이조절
#     src_corpus = []
#     tgt_corpus = []

#     for s, t in zip(src, tgt):
#         s = preprocess_sentence(s)
#         t = preprocess_sentence(t)
#         s = (mecab.morphs(s))
#         print(s)
#         t = (mecab.morphs(t))
            
#         if len(s) < 40 and len(t) < 40:
#             src_corpus.append(s)
#             tgt_corpus.append(t)
            
# #             src_corpus.append(preprocess_sentence(s))
# #             tgt_corpus.append(preprocess_sentence(t))            
            
# #             cleaned_src = preprocess_sentence(s)
# #             cleaned_tgt = preprocess_sentence(t)

#     # 중복제거        
#     que_corpus = []
#     ans_corpus = []
#     s_dic = {}
# #     t_dic = {}
#     for s, t in zip(src_corpus, tgt_corpus):
#         if s not in s_dic:
#             s_dic[s] = 1
# #             s = mecab.morphs(s)
# #             t = mecab.morphs(t)
#             que_corpus.append(s)
#             ans_corpus.append(t)
            
# #         if t not in t_dic:
# #             t_dic[t] = 1
# #             que_corpus.append(s)
# #             ans_corpus.append(t)

#     return que_corpus, ans_corpus
            

In [11]:
from konlpy.tag import Mecab
mecab = Mecab()

def build_corpus(src, tgt):

    src_corpus = []
    tgt_corpus = []

    s_dic = {}
#     t_dic = {}
    for s, t in zip(src, tgt):
        if s not in s_dic:
            s_dic[s] = 1

            src_corpus.append(s)
            tgt_corpus.append(t)    
    
    
    que_corpus = []
    ans_corpus = []

    for s, t in zip(src_corpus, tgt_corpus):
        if len(s) < 40 and len(t) < 40:
#             src_corpus.append(preprocess_sentence(s))
#             tgt_corpus.append(preprocess_sentence(t))            
            
# mecab없는용/ 필요하면 삭제 후 아래 사용
            que_corpus.append(preprocess_sentence(s))
            ans_corpus.append(preprocess_sentence(t))
            
# 아래에서 mecab사용해야해서 주석처리. 이부분 주석 풀어서 사용하면 됌
#             cleaned_src = preprocess_sentence(s)
#             cleaned_tgt = preprocess_sentence(t)
#             que_corpus.append(mecab.morphs(cleaned_src))
#             ans_corpus.append(mecab.morphs(cleaned_tgt))
            


    return que_corpus, ans_corpus
            

In [12]:
que_corpus, ans_corpus = build_corpus(questions, answers)

In [13]:
print(que_corpus[0], ans_corpus[0])

12시 땡! 하루가 또 가네요.


### 여기까지 데이터 전처리

## 12-5. 데이터 부풀리기

In [14]:
import os
import gensim

path = os.getenv("HOME")+'/aiffel/transformer_chatbot/'
wv = gensim.models.Word2Vec.load(path+'ko.bin')

In [15]:
wv.most_similar('바나나')

/home/aiffel-dj62/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('코코넛', 0.8097119331359863),
 ('시금치', 0.7701147794723511),
 ('레몬', 0.7688493728637695),
 ('땅콩', 0.7684734463691711),
 ('파인애플', 0.7639915943145752),
 ('녹차', 0.7631460428237915),
 ('딸기', 0.7617197036743164),
 ('바닐라', 0.7497864365577698),
 ('파슬리', 0.7447543144226074),
 ('코코아', 0.7408243417739868)]

In [16]:
# 단어단위로 쪼개서 순서대로 wv.most_similar과 같은 단어를 출력

import random

sample_sentence = "그거 알아? 바나나만 있어"
sample_tokens = mecab.morphs(sample_sentence)

selected_tok = random.choice(sample_tokens)
print(selected_tok)
result = ""
for tok in sample_tokens:
    if tok is selected_tok:
        print(tok)
        result += wv.most_similar(tok)[0][0] + " "

    else:
        result += tok + " "

print("From:", sample_sentence)
print("To:", result)

알
알
From: 그거 알아? 바나나만 있어
To: 그거 울 아 ? 바나나 만 있 어 


/home/aiffel-dj62/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [17]:
print(len(que_corpus))

11545


In [18]:
def lexical_sub(que, ans, word2vec):
    import random
    
    # 빈 문자열 선언
    que_res = ""
    ans_res = ""
    
    # 형태소 분석 후 토큰화된 문장을 저장
    que_toks = mecab.morphs(que)
    ans_toks = mecab.morphs(ans)
#     print(que_toks, ans_toks)
    # 문장의 단어 하나를 랜덤하게 뽑아서 가장 유사한 단어로 변환
    # 입력받는 질문과 응답 답변 모두 저장
    try:
        _from_que = random.choice(que_toks)
        _to_que = word2vec.most_similar(_from_que)[0][0]
#         print(_from_que, _to_que)
        
        _from_ans = random.choice(ans_toks)
        _to_ans = word2vec.most_similar(_from_ans)[0][0]
#         print(_from_ans, _to_ans)

    except:   # 단어장에 없는 단어
        return [], []# 두개를 받아서 빈 리스트 두개를 리턴
#     print("===============================")
    
    for tok in que_toks:
        if tok is _from_que: que_res += _to_que + " "# 하나 랜덤하게 뽑아서 해당 단어와 가장 유사한 단어로 바꿔주는 for문
        else: que_res += tok + " "
            
    for tok in ans_toks:
        if tok is _from_ans: ans_res += _to_ans + " "# 하나 랜덤하게 뽑아서 해당 단어와 가장 유사한 단어로 바꿔주는 for문
        else: ans_res += tok + " "
    print(que, que_res, ans, ans_res)
    
    return (que, que_res), (ans, ans_res)

In [19]:
a, b = lexical_sub("12시 땡!", "하루가 또 가네요", wv)
print(a)
print(b)

12시 땡! 12 시 끗 !  하루가 또 가네요 하루 가 각기 가 네요 
('12시 땡!', '12 시 끗 ! ')
('하루가 또 가네요', '하루 가 각기 가 네요 ')


/home/aiffel-dj62/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  app.launch_new_instance()
/home/aiffel-dj62/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


In [20]:
for i in range(len(a)):
    print(a[i])

12시 땡!
12 시 끗 ! 


## 토크나이저도 만들어야함

In [21]:
from tqdm import tqdm_notebook

new_que_corpus = []
new_ans_corpus = []

for idx in tqdm_notebook(range(len(que_corpus))):
    old_que = (que_corpus[idx])
    old_ans = (ans_corpus[idx])
    
    
    new_que, new_ans = lexical_sub(old_que, old_ans, wv)
#     print(new_que, new_ans)
    for i in range(len(new_que)):
        if new_que is not [] and new_ans is not []:
            new_que_corpus.append(mecab.morphs(new_que[i]))
            new_ans_corpus.append(mecab.morphs(new_ans[i]))
        
#     new_que_corpus.append(old_que)
#     if new_ans is not []: 
#     new_ans_corpus.append(old_ans)
#     new_corpus.append(old_src)

# print(new_que_corpus[:10])

/home/aiffel-dj62/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/11545 [00:00<?, ?it/s]

/home/aiffel-dj62/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  app.launch_new_instance()
/home/aiffel-dj62/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


12시 땡! 12 시 땡 캐치  하루가 또 가네요. 하루 놀드 또 가 네요 . 
3박4일 놀러가고 싶다 3 박 4 일 놀 ㄹ래 가 고 싶 다  여행은 언제나 좋죠. 여행 은 항상 좋 죠 . 
ppl 심하네 ppl 강하 네  눈살이 찌푸려지죠. 눈살 이 찌푸려 꼼짝 죠 . 
sd카드 안돼 sd 단말기 안 돼  다시 새로 사는 게 마음 편해요. 다시 새로 사 는 게 마음 편해요 는데 
sns 맞팔 왜 안하지 sns 맞 팔 과연 안 하 지  잘 모르고 있을 수도 있어요. 많이 모르 고 있 을 수 도 있 어요 . 
sns 시간낭비인 거 아는데 매일 하는 중 sns 시간 소모 인 거 아 는데 매일 하 는 중  시간을 정하고 해보세요. 시간 를 정하 고 해 보 세요 . 
sns 시간낭비인데 자꾸 보게됨 sns 시간 낭비 인데 그럼 보 게 됨  시간을 정하고 해보세요. 시간 을 정하 기에 해 보 세요 . 
가끔 궁금해 가끔 궁금 해의  그 사람도 그럴 거예요. 그 사람 도 그럴 것 예요 . 
가끔 뭐하는지 궁금해 가끔 뭐 시키 는지 궁금 해  그 사람도 그럴 거예요. 그녀 사람 도 그럴 거 예요 . 
가끔은 혼자인게 좋다 가끔 은 혼자 인은 게 좋 다  혼자를 즐기세요. 혼자 을 즐기 세요 . 
가만 있어도 땀난다 가만 있 더라도 땀 난다  땀을 식혀주세요. 땀 을 식혀 주 세요 는데 
가상화폐 쫄딱 망함 가상현실 화폐 쫄딱 망함  어서 잊고 새출발 하세요. 어서 잊 고 새 출항 하 세요 . 
가스불 켜고 나갔어 가스 불 켜 기에 나갔 어  빨리 집에 돌아가서 끄고 나오세요. 빨리 집 에 돌아가 서 끄 고 나오 세요 는데 
가스불 켜놓고 나온거 같아 가스 불 켜 놓 기에 나온 거 같 아  빨리 집에 돌아가서 끄고 나오세요. 빨리 집 에 돌아가 서 끄 기에 나오 세요 . 
가스비 너무 많이 나왔다. 가스 비 너무 많이 나왔 다 는데  다음 달에는 더 절약해봐요. 다음 달이 에 는 더 절약 해 봐요 . 
가스비 비싼데 감기 걸리겠어 가스 비 비싼데 감기 매달리 겠 어  따뜻하

고백하고 후회하면 어떡하지 고백 하 기에 후회 하 면 어떡 하 지  후회는 후회를 낳을뿐이에요. 용기 내세요. 후회 는 후회 를 낳 를 뿐 이 에요 . 용기 내 세요 . 
고시원 답답해 고시원 지루 해  돈을 모아서 다른 곳으로 이사갈 수 있을 거예요. 돈 을 모아서 다른 곳 으로 이사 갈 수 있 을 거 예요 는데 
고시원에서 나가고 싶어 고시원 에서 나가 고 싶 어서  더 좋은 곳에서 살 수 있을 거예요. 더 좋 은 곳 오세아니아 살 수 있 을 거 예요 . 
고시원에서 탈출하고 싶어 고시원 에서 탈출 시키 고 싶 어  더 좋은 곳에서 살 수 있을 거예요. 더 괜찮 은 곳 에서 살 수 있 을 거 예요 . 
고양이 동영상 보는 중 토끼 동영상 보 는 중  완전 귀엽죠? 완전 귀엽 지요 ? 
고양이 키우고 싶어 고양이 키우 고 겠 어  자신을 먼저 키우세요. 자기 을 먼저 키우 세요 . 
고의는 아닌데 실수를 한 거 같아 악의 는 아닌데 실수 를 한 거 같 아  용서를 구하세요. 용서 을 구하 세요 . 
고집 센 사람 고집 센 젊은이  피할 수 있으면 피하세요. 피할 수 있 으면 피하 세요 는데 
골프 못 치는데 테니스 못 치 는데  처음부터 잘하는 사람은 없어요. 본격적 부터 잘 하 는 사람 은 없 어요 . 
골프 배워야 돼 테니스 배워야 돼  시간내서 가보세요. 시간 내 불쌍히 가 보 세요 . 
골프치러 가야돼 골프 치러 놀드 야 돼  시간내서 가보세요. 분간 내 서 가 보 세요 . 
곱창 먹고 싶어. 곱창 씹 고 싶 어 .  미리 미리 충전해주세요. 미리 미리 충전 해 주 ㅂ시오 . 
공무원 괜찮겠지 공무원 좋 겠 지  안정적이고 좋죠. 안정 적 그러 고 좋 죠 . 
공무원 되고 싶다 공무원 되 고 겠 다  준비해보세요. 준비 해 보 ㅂ시오 . 
공무원 되면 좋겠다 공무원 되 ㄴ다면 좋 겠 다  준비해보세요. 준비 해 보 세요 는데 
공무원 시험 공부 힘들다 공무원 시험 가르치 힘들 다  합격 기원해요! 합격 기원전 해요 ! 
공무원 시험 죽을 거 같아 공무

...

좋아하는 선배가 있는데 선톡오게 하는 방법 좀. 좋 아 하 는 선배 놀드 있 는 데 선 톡 오 게 하 는 방법 좀 .  먼저 연락해서 대화를 이어나가는게 더 좋습니다. 먼저 연락 해서 대화 를 이 어서 나가 는 게 더 좋 습니다 . 
좋아하는 앞에서 방구를 꼈어. 좋 아 하 는 앞 오세아니아 방구 를 꼈 어 .  생각만해도 소름돋네요. 생각 만 해도 소름 돋 네요 는데 
좋아하는 애 가망 없으면 포기하는게 맞을까요? 좋 아 하 는 애 승산 없 으면 포기 하 는 게 맞 을까요 ?  가망이 없다면 정리하는게 덜 아플 거예요. 가망 그러 없 다면 정리 하 는 게 덜 아플 거 예요 . 
좋아하는 애 상메가 무슨 뜻인지 모르겠어. 좋 아 하 는 애 상 메 가 무슨 뜻 인지 모르 겠 어서 .  궁금할만 해요. 궁금하 할 만 해요 . 
좋아하는 애 피하는 방법 조언 좀. 좋 아 하 는 애 없애 는 방법 조언 좀 .  동선을 어느정도 파악하는게 도움이 될거예요. 동선 을 어느 정도 파악 하 는 게 영향 이 될 거 예요 . 
좋아하는 애 피하는 중임. 좋 아서 하 는 애 피하 는 중임 .  마음을 정리하려나봐요. 마음 을 계산 하 려나 봐요 . 
좋아하는 애 피하면서도 우울해. 좋 아서 하 는 애 피하 면서 도 우울 해 .  당당히 나서지 못하는 아쉬움이겠지요. 당당히 나서 지 못하 는 아쉬움 이 겠 지요 는데 
피해 다니면 좋아했던 마음이 좀 사라질까? 피해 돌아다니 면 좋 아 했 던 마음 이 좀 사 라 질까 ?  한동안은 힘들겠지만 서서히 사라질 거예요. 한동안 은 힘들 싶 지만 서서히 사라질 거 예요 . 
좋아하는 애한테 페메 해볼까? 좋 아 하 는 애 한테 페메 해 볼까 ...  직접 연락하는 게 좋을 것 같아요. 직접 연락 하 ㄴ다는 게 좋 을 것 같 아요 . 
좋아하는 애가 여친 사귈 마음이 없대. 좋 아 하 는 애 가 여친 사귈 괴로움 이 없 대 .  거절의 뜻으로 받아들여야할지도 모르겠네요. 거절 의 뜻 으로서 받아들여야 할지 도 모르 겠 네요 . 
좋아하는 애

좋아하면 이용당해도 되는거야? 좋 아 시키 면 이용 당해도 되 는 거 야 ?  절대 안됩니다. 절대 안 됩니다 는데 
좋아하면서 너무 싫어. 괜찮 아 하 면서 너무 싫 어 .  사랑은 복합적인 감정이니까요. 슬픔 은 복합 적 인 감정 이 니까요 . 
좋아하면서도 너무 싫은 감정 뭘까요. 좋 아서 하 면서 도 너무 싫 은 감정 뭘까요 .  좋아하는 만큼 지쳐가는 마음 아닐까요. 괜찮 아 하 는 만큼 지쳐 가 는 마음 아닐까요 . 
좋아하지 않는데 계속 연락하고 만나는거 가능한가. 좋 아 하 지 않 는데 계속 연락 하 고 만나 는 것 가능 한가 .  가벼운 친구사이로 생각할 수도 있죠. 가벼운 친구 간의 로 생각 할 수 도 있 죠 . 
좋아해 괜찮 아 해  고백하세요. 회개 하 세요 . 
좋아했던 기억 영원히 없앨 수는 없을까. 좋 아 했 던 기억 영원히 없앨 수 ㄴ다는 없 을까 .  없애는 게 아니라 무덤덤해지는 걸 거예요. 없애 는 게 아니 라 무덤덤 해의 지 는 걸 거 예요 . 
좋아했던 사람 영원히 잊을 수 없을까? 좋 아 했 던 사람 영원히 잊 을 수 없었 을까 ?  시간이 좀 걸리겠지만 가능할 거예요. 시간 이 좀 걸리 싶 지만 가능 할 거 예요 . 
좋아했던 사람도 금방 잊을 수 있을까. 좋 아 했 더 사람 도 금방 잊 을 수 있 을까 .  아무래도 시간이 필요하겠죠. 아무래도 시간 이 필요 하 겠 죠 는데 
좋아했던 사람에 대한 트라우마 때문에 연애를 못하겠어. 좋 아 했 던 젊은이 에 대한 트라우마 때문 에 연애 를 못 하 겠 어 .  트라우마를 없앨 사람이 나타날 거예요. 트라우마 을 없앨 사람 이 나타날 거 예요 . 
좋아했지만 고백은 못하겠어. 좋 아서 했 지만 고백 은 못 하 겠 어 .  애틋한 사랑이네요. 애틋 한 슬픔 이 네요 . 
좋아했지만 끝내 고백은 못할 듯. 괜찮 아 했 지만 끝내 고백 은 못 할 듯 .  애틋한 사랑이네요. 애틋 한두 사랑 이 네요 . 
좋은 곳 오면 생각나는 사람 좋 은 곳 오 ㄴ다면 생각나 는 사람  그게 진

짝남이 좋아하는 사람이 있대요. 짝 남 그러 좋 아 하 는 사람 이 있 대요 .  저까지 마음이 안좋네요. 저 까지 마음 이 안 좋 군요 . 
짝남이 좋아하는애 있거나 여친 있으면 포기하는 이유가 뭐야? 짝 남 이 좋 아 하 는 사우스캐롤라이 있 거나 여친 있 으면 포기 하 는 이유 가 뭐 야 ?  그 사람에 대한 예의일 것 같아요. 그녀 사람 에 대한 예의 일 것 같 아요 . 
짝남이 학교에서 평이 안 좋아. 짝 남 이 학교 에서 평 이 못가 좋 아 .  무슨 이유가 있을 거예요. 무슨 이유 가 있 을 것 예요 . 
짝남이 헤어진지 별로 안됐는데 다가가도 될까? 짝 남 이 헤어진지 별로 안 됐 으며 다가가 도 될까 ?  위로를 해줘보세요. 위로 를 해 줘 살펴보 세요 . 
짝남이든 썸남이든 뭐든 있었음 좋겠다. 짝 남 이 든 썸남 이 든 뭐 든 있 었 음 괜찮 겠 다 .  곧 사랑이 찾아올거라 믿어요. 곧 사랑 이 찾아올 거 라 믿 어요 는데 
짝남이랑 같이 도서관 가고 싶어. 방법 좀. 짝 남 이랑 같이 도서관 가 고 싶 어 . 방식 좀 .  공부 도와달라고 해보세요. 공부 도와 달 라고 해 보 ㅂ시오 . 
짝남이랑 괜히 술 마셨어. 짝 남 이랑 괜히 술 마셨 어서 .  항상 아쉬움이 남는가봐요. 항상 아쉬움 이 남 습니까 봐요 . 
짝남이랑 끝났는데 그래도 매일 볼 수 있어서 좋아. 짝 남 이랑 끝 났 으며 그래도 매일 볼 수 있 어서 좋 아 .  정말 사랑하는 분인가봐요. 정말 슬픔 하 는 분 인가 봐요 . 
짝남이랑 동생 이름이 똑같아. 쌍 남 이랑 동생 이름 이 똑같 아 .  놀랐겠어요. 놀랐 겠 어요 는데 
짝남이랑 사귀기로 했어 짝 남의 이랑 사귀 기 로 했 어  축하드려요. 축하 드려요 는데 
짝남이랑 술 마셨는데 기분만 더 더러워졌어. 짝 남 아줌마 술 마셨 는데 기분 만 더 더러워졌 어 .  항상 후회만 남나봐요. 언제나 후회 만 남 나 봐요 . 
짝남이랑 썸 탈 것 같애. 짝 남 이랑 썸 탈 것 같 사우스캐롤라이 .  좋은 결실 맺길 바랍

짝녀한테 휘둘리고 싶지 않은데 이러는 내가 싫다. 짝 녀 한테 휘둘리 고 싶 지 못하 은데 이러 는 내 가 싫 다 .  자신의 의지를 굳건히 하세요. 자신 의 의지 를 굳건히 시키 세요 . 
짝녀한테도 철벽쳐요. 성격이 왜 그럴까요. 짝 녀 한테 도 철벽 쳐요 는데 성격 이 왜 그럴까요 는데  두려운게 있는게 아닐까요. 두려운 게 있 는 도록 아닐까요 . 
짝사랑 그만하고 싶다 짝사랑 그만 하 기에 싶 다  짝사랑은 시작과 끝을 자신이 결정할 수 있어요. 짝사랑 은 시작 과 끝 을 자신 이 결정 할 수 없 어요 . 
짝사랑 그만할래 그리워하 그만 할래  짝사랑은 시작과 끝을 자신이 결정할 수 있어요. 짝사랑 은 시작 과 끝 을 자신 이 결정 할 수 있 어요 는데 
짝사랑 끝내고 나니 마음이 공허하다. 짝사랑 끝내 고 나 니 마음 이 고독 하 다 .  사랑했던 시간이 멈춰버려서 그런거예요. 사랑 했 던 시간 그러 멈춰 버려서 그런 거 예요 . 
짝사랑 중이었는데 솔로 탈출했어요. 짝사랑 중 이 였 는데 솔로 탈출 했 어요 .  좋은 소식이네요. 좋 은데 소식 이 네요 . 
짝사랑 중인 내가 이해가 안돼. 그리워하 중 인 내 가 이해 가 안 돼 .  짝사랑 앞에 장사 없지요. 짝사랑 옆 에 장사 없 지요 . 
짝사랑 중인데 나혼자 영화찍어. 짝사랑 중의 인데 나 혼자 영화 찍 어 .  짝사랑이 그런가봐요. 짝사랑 그러 그런가 봐요 . 
짝사랑 중인데 나혼자 치정 로맨스야. 짝사랑 중의 인데 나 혼자 치정 로맨스 야 .  짝사랑이 그렇게 만드네요. 짝사랑 그러 그렇게 만드 네요 . 
짝사랑 중일 때 장점. 짝사랑 중 월과 때 장점 .  짝사랑 상대 생각에 설레는 것. 짝사랑 상대 생각 이에 설레 는 것 . 
짝사랑 진짜 못해먹겠다. 짝사랑 진짜 못하 해 먹 겠 다 .  지쳤나봐요. 지쳤 나 봐요 는데 
짝사랑 포기하는 것도 힘드네. 짝사랑 포기 하 는 것 때문 힘드 네 .  사랑이었으니까요. 사랑 그러 었 으니까요 . 
짝사랑 하는 남자 때문에 하루에도 몇 번 기분

친한 동생인데 짝남이 되고 있는 느낌이에요. 친한 동생 인데 짝 남 이 되 고 있 는 웃음 이 에요 .  받아들이기 쉽지 않죠. 받아들이 기 쉽 지 못하 죠 . 
친한 동생인데 짝녀가 된 것 같아요. 친한 동생 인데 쌍 녀 가 된 것 같 아요 .  있을 수 있는 일이에요. 있 를 수 있 는 일 이 에요 . 
친한 동생인데 짝사랑하는 것 같습니다. 친한 동생 인데 짝사랑 하 는 것 같 네요 .  남녀 사이에서 있을 수 있는 일입니다. 남녀 사이 에서 있 을 수 있 는 월과 입니다 . 
친한 친구가 저를 좋아하는 것 같아요. 친한 친구 가 저 를 좋아하 는 것 똑같 아요 .  친구사이가 깨질까 걱정되나봐요. 친구 사이 가 깨뜨리 질까 걱정 되 나 봐요 . 
친한 친구의 구여친이랑 사귀어도 되나. 친한 친구 의 구로 여친 이랑 사귀 어도 되 나 .  친구라면 많이 실망할겁니다. 친구 라면 많이 당황 할 겁니다 . 
카리스마 있는 사람이 좋아 카리스마 있 는 사람 이 좋 아서  당신을 이끌어주는 남자를 만날 수 있을 거예요. 당신 을 이끌 어서 주 는 남자 를 만날 수 있 을 거 예요 . 
카톡하는데 자꾸 웃음 나. 카톡 하 는데 그럼 웃음 나 .  부러워요. 자연스러운 현상이에요. 부러워요 . 자연 스러운 현상 이 ㄴ가요 . 
카톡할때 너무 칼답하면 부담스러워할까? 카톡 할 때 워낙 칼 답 하 면 부담 스러워 할까 ?  귀여울 거 같아요. 귀여울 거 똑같 아요 . 
카페 알바생이 좋아졌는데 들이대도 괜찮을까? 카페 알바 생 이 괜찮 아 졌 는데 들이대 도 괜찮 을까 ?  언제 끝나냐고 물어보세요. 언제 종료 냐고 물 어 보 세요 . 
카페에서 같이 알바하는 사람이 좋아졌어요. 카페 에서 같이 알 바 하 는 사람 이 좋 아서 졌 어요 .  자연스럽게 말을 걸어보세요. 자연 스럽 게 말 을 걸 어 살펴보 세요 . 
커뮤니티 하는 썸녀 어때? 커뮤니티 하 는 썸녀 어때 ...  그게 무슨 문제인가요. 그게 어찌 문제 인가요 . 
커플 여행 가면 잘 싸워? 커플 항해 가 

In [22]:
print(len(new_que_corpus))

17430


In [23]:
print(new_que_corpus[:10])

[['12', '시', '땡', '!'], ['12', '시', '땡', '캐치'], ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'], ['3', '박', '4', '일', '놀', 'ㄹ래', '가', '고', '싶', '다'], ['ppl', '심하', '네'], ['ppl', '강하', '네'], ['sd', '카드', '안', '돼'], ['sd', '단말기', '안', '돼'], ['sns', '맞', '팔', '왜', '안', '하', '지'], ['sns', '맞', '팔', '과연', '안', '하', '지']]


## Step 5. 데이터 벡터화


In [111]:
_enc = []
_dec = []


for i in new_que_corpus:
    _enc.append(i)
#     _enc.append(["<start>"] + k + ["<end>"])
for k in new_ans_corpus:
    _dec.append(["<start>"] + k + ["<end>"])
    
print(_enc[:5], _dec[:5])

[['12', '시', '땡', '!'], ['12', '시', '땡', '캐치'], ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'], ['3', '박', '4', '일', '놀', 'ㄹ래', '가', '고', '싶', '다'], ['ppl', '심하', '네']] [['<start>', '하루', '가', '또', '가', '네요', '.', '<end>'], ['<start>', '하루', '놀', '드', '또', '가', '네요', '.', '<end>'], ['<start>', '여행', '은', '언제나', '좋', '죠', '.', '<end>'], ['<start>', '여행', '은', '항상', '좋', '죠', '.', '<end>'], ['<start>', '눈살', '이', '찌푸려', '지', '죠', '.', '<end>']]


In [112]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, tokenizer

In [113]:
# len(_enc+_dec)

In [114]:
total_tensor, total_tokenizer = tokenize(_enc+_dec)
que_tensor, que_tokenizer = tokenize(_enc)
ans_tensor, ans_tokenizer = tokenize(_dec)

que_train, ans_train = que_tensor, ans_tensor

print("Total Vocab Size:", len(total_tokenizer.index_word))
print("Question Vocab Size:", len(que_tokenizer.index_word))
print("Answer Vocab Size:", len(ans_tokenizer.index_word))

Total Vocab Size: 6865
Question Vocab Size: 5273
Answer Vocab Size: 4090


In [115]:
que_tensor[1]

array([1510,  647, 2836,  648,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0], dtype=int32)

In [116]:
enc_train, enc_val, dec_train, dec_val = \
train_test_split(que_tensor, ans_tensor, test_size=0.01)

print("enc_train :", len(enc_train), "enc_val :", len(enc_val))
print("dec_train :", len(dec_train), "dec_val :",len(dec_val))

enc_train : 17255 enc_val : 175
dec_train : 17255 dec_val : 175


transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)


In [117]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

In [118]:
# Generate Padding Mask
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [119]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        self.depth = d_model // self.num_heads

        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)

        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions


    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x


    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)

        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)

        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)

        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights

In [120]:
# Position-wise Feed Forward Network 구현
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)

        return out

In [121]:
# Encoder layer

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):

        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, enc_attn

In [122]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, enc_out, causality_mask, padding_mask):

        """
        Masked Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        """
        Multi-Head Attention
        """
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.dec_self_attn(out, enc_out, enc_out, causality_mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [123]:
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):
        out = x

        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)

        return out, enc_attns

In [124]:
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]


    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x

        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [125]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out


    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)

        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, causality_mask, dec_mask)

        logits = self.fc(dec_out)

        return logits, enc_attns, dec_attns, dec_enc_attns

In [126]:
VOCAB_SIZE = len(total_tokenizer.index_word)

In [185]:
transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)

d_model = 512

In [186]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=2000):
        super(LearningRateScheduler, self).__init__()

        self.d_model = d_model
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)

        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [187]:
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)


In [188]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


In [189]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


In [190]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [191]:
from tqdm import tqdm_notebook 

BATCH_SIZE = 64
EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                    dec_train[idx:idx+BATCH_SIZE],
                    transformer,
                    optimizer)

        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

/home/aiffel-dj62/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/270 [00:00<?, ?it/s]

...

  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/270 [00:00<?, ?it/s]

In [192]:
sentence = preprocess_sentence("치킨을 먹었더니 속이 안좋네요")
sentence = mecab.morphs(sentence)
tokens = total_tokenizer.texts_to_sequences(sentence)# 아이디로 토큰화
# pieces = total_tokenizer.sequences_to_texts(tokens)# 문자로 토큰화
tokens

[[2827], [7], [87], [58], [582], [335], [4], [43], [12], [41]]

In [193]:
pieces = total_tokenizer.sequences_to_texts(tokens)# 문자로 토큰화
pieces

['치킨', '을', '먹', '었', '더니', '속', '이', '안', '좋', '네요']

In [194]:
# translate()
# 전처리부터 모든 단계를 포함하고 있어야 함

def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
    sentence = preprocess_sentence(sentence)
    sentence = mecab.morphs(sentence)
    tokens = total_tokenizer.texts_to_sequences(sentence)# 아이디로 토큰화
    pieces = total_tokenizer.sequences_to_texts(tokens)# 문자로 토큰화
    
    
    _input = tf.keras.preprocessing.sequence.pad_sequences(tokens,
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    
    ids = []
    output = tf.expand_dims([1], 0)
#     print(output)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()
        print(predicted_id)
        if predicted_id == 2:
            print(ids)
            result = total_tokenizer.sequences_to_texts(ids)

            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append([predicted_id])
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = total_tokenizer.sequences_to_texts(ids)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

def translate(sentence, model, src_tokenizer, tgt_tokenizer):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model, src_tokenizer, tgt_tokenizer)

    return result
print("슝=3")

슝=3


In [195]:
'''
# 예문
1. 지루하다, 놀러가고 싶어.
2. 오늘 일찍 일어났더니 피곤하다.
3. 간만에 여자친구랑 데이트 하기로 했어.
4. 집에 있는다는 소리야.'''

'\n# 예문\n1. 지루하다, 놀러가고 싶어.\n2. 오늘 일찍 일어났더니 피곤하다.\n3. 간만에 여자친구랑 데이트 하기로 했어.\n4. 집에 있는다는 소리야.'

In [196]:
translate("지루하다, 놀러가고 싶어.", transformer, total_tokenizer, total_tokenizer)

337
26
106
226
25
18
22
2
[[337], [26], [106], [226], [25], [18], [22]]


['처럼', '겠', '알', '봅니다', '에', '지', '도']

In [197]:
translate("오늘 일찍 일어났더니 피곤하다.", transformer, total_tokenizer, total_tokenizer)

65
270
65
2
[[65], [270], [65]]


['남자', '인지', '남자']

In [198]:
translate("간만에 여자친구랑 데이트 하기로 했어.", transformer, total_tokenizer, total_tokenizer)

99
34
1636
5
3
2
[[99], [34], [1636], [5], [3]]


['못', '같', '힘듭니다', '하', '<end>']

In [199]:
translate("집에 있는다는 소리야.", transformer, total_tokenizer, total_tokenizer)

325
3
2
[[325], [3]]


['몰라요', '<end>']

평가문항
상세기준
1. 챗봇 훈련데이터 전처리 과정이 체계적으로 진행되었는가?


챗봇 훈련데이터를 위한 전처리와 augmentation이 적절히 수행되어 3만개 가량의 훈련데이터셋이 구축되었다.

2. transformer 모델을 활용한 챗봇 모델이 과적합을 피해 안정적으로 훈련되었는가?

과적합을 피할 수 있는 하이퍼파라미터 셋이 적절히 제시되었다.

3. 챗봇이 사용자의 질문에 그럴듯한 형태로 답하는 사례가 있는가?

주어진 예문을 포함하여 챗봇에 던진 질문에 적절히 답하는 사례가 제출되었다.

## 회고

노드를 진행하면서 굉장히 오랜 시간이 걸렸습니다. 모델 구현부분은 그대로 가져다 쓰면 됐지만 전처리작업과 평가부분에서 건드려야 할 부분이 많았습니다. 하이퍼 파라미터를 바꿔 보는것은 큰 도움이 되진 않는것 같고 데이터가 충분하지 않아 제대로 된 번역이 되지 않는것 같습니다.